In [ ]:
# Imports
import pandas as pd
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
pio.renderers.default = "notebook_connected"


# Analyse de l'évolution des températures en France (1950-1974)

**Problématique :** Visualiser l'accélération du réchauffement climatique en France métropolitaine de 1950 à 1974 : des moyennes annuelles aux réalités locales.

Ce notebook utilise des données issues de la réanalyse ERA5 (Copernicus), masquées pour ne conserver que la France métropolitaine. 


# Chargement des données

In [ ]:
# Chargement du CSV (Historique Temporel)
df = pd.read_csv("donnees_meteo_france_25ans.csv")
df['time'] = pd.to_datetime(df['time'])
df['Year'] = df['time'].dt.year
df['Month'] = df['time'].dt.month

# Chargement du NetCDF (Carte)
try:
    ds_map = xr.open_dataset("carte_exemple_25ans.nc")
    # Conversion en DataFrame pour Plotly
    df_map = ds_map.to_dataframe().reset_index().dropna()
    # On s'assure d'avoir l'année pour l'animation
    if 'time' in df_map.columns:
        df_map['Year'] = df_map['time'].dt.year
    if df_map['t2m'].mean() > 200:
        df_map['Temperature_C'] = df_map['t2m'] - 273.15
    else:
        df_map['Temperature_C'] = df_map['t2m']
except Exception as e:
    print(f"⚠️ Attention : Le fichier carte .nc n'a pas pu être chargé ({e})")
    df_map = None

# Courbe Anomalie Température France Métropolitaine (1950-1974)



In [ ]:
# %pip install nbformat

# On calcule une "moyenne de référence" (sur les 5 premières années dispos par exemple)
annee_min = df['Year'].min()
ref_mean = df[(df['Year'] >= annee_min) & (df['Year'] <= annee_min + 10)]['Temperature_C'].mean()
df['Anomalie'] = df['Temperature_C'] - ref_mean

# On agrège par an pour avoir une courbe propre
df_yearly = df.groupby('Year')[['Temperature_C', 'Anomalie']].mean().reset_index()

fig1 = px.bar(df_yearly, x='Year', y='Anomalie', color='Anomalie',
              title=f"Anomalie de température en France (Réf: {annee_min}-{annee_min+10})",
              labels={'Anomalie': 'Écart à la moyenne (°C)'},
              color_continuous_scale='RdBu_r', # Bleu = Froid, Rouge = Chaud
              range_color=[-2, 2]) # Fixe l'échelle pour bien voir les couleurs

# Ajout d'une ligne de tendance (Moyenne mobile)
fig1.add_scatter(x=df_yearly['Year'], y=df_yearly['Anomalie'].rolling(10).mean(),
                 mode='lines', name='Tendance (10 ans)', line=dict(color='black', width=3))

fig1.add_hline(y=0, line_dash="dash", line_color="grey")
fig1.show()

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.21.2-py3-none-any.whl (24 kB)
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)

   ----------- ---------------------------- 2/7 [attrs]
   ----------------- ---------------------- 3/7 [referencing]
   ---------------------- ----------------- 4/7 [jsonschema-specifications]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------------- ----- 6/7 [nbformat]
   ---------------------------------- ----- 6/7 [nbformat]
   ---------------------------------------- 7/7 [nbformat]

Note: you may need to restart the kernel to use updated packages.


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed